In [1]:
# !pip install spacy

In [2]:
import os 
import pandas as pd 
import numpy as np
from tqdm import tqdm
# from google.colab import files

In [3]:
# !wget https://cs.stanford.edu/people/karpathy/deepimagesent/caption_datasets.zip

In [4]:
# !unzip caption_datasets.zip -d caption_datasets

In [5]:
dfjson = pd.read_json('/mnt/nas2/GrimaRepo/datasets/flickr30k/annotations/dataset_flickr30k.json')


list_df = [('image_ID', 'caption_ID', 'caption', 'split')]

for k_img in range(len(dfjson['images'])):
    dict_img = dfjson['images'][k_img]
    imgid = dict_img['filename'].split('.')[0]
    split = dict_img['split']
    text = dict_img['sentences']
    # for each caption
    for capt in text:
        # IID, CID, caption, split
        list_df.append((imgid, capt['sentid'], capt['raw'], split))
        
df = pd.DataFrame(list_df[1:], columns=list_df[0])#.set_index(['IID', 'CID'])
df

,image_ID,caption_ID,caption,split
0,1000092795,0,Two young guys with shaggy hair look at their ...,train
1,1000092795,1,"Two young, White males are outside near many b...",train
2,1000092795,2,Two men in green shirts are standing in a yard.,train
3,1000092795,3,A man in a blue shirt standing in a garden.,train
4,1000092795,4,Two friends enjoy time spent together.,train
...,...,...,...,...
155065,998845445,155065,A man in shorts and a Hawaiian shirt leans ove...,train
155066,998845445,155066,"A young man hanging over the side of a boat, w...",train
155067,998845445,155067,A man is leaning off of the side of a blue and...,train
155068,998845445,155068,"A man riding a small boat in a harbor, with fo...",train


In [6]:
# df.to_csv('Caption_all.tsv', sep='\t', index=False)
# files.download('Caption_all.tsv')

# Skills

In [7]:
from skillschecker import skillsChecker

In [8]:
checker = skillsChecker()
list_captions = df.caption.values

split = 'val'
skill = 'gender' # color, gender, counting

dft = df[df.split == split]

In [9]:
list_tuple_new_captions, wds = checker.change_captions_skill(list_captions=dft.caption, skill=skill, verbose=True)

100%|██████████| 5070/5070 [00:00<00:00, 7451.65it/s]
2922it [00:01, 1966.77it/s]


In [10]:
list_tuple_new_captions, wds = checker.change_captions_skill(list_captions=dft.caption, skill=skill, verbose=True)

capt_selected = [k[0] for k in list_tuple_new_captions]
dftskill = dft[dft.caption.isin(capt_selected)]

100%|██████████| 5070/5070 [00:00<00:00, 7732.38it/s]
2922it [00:01, 1979.19it/s]


In [11]:
dftskill['word_detected'] = wds
dftskill['augmented_captions'] = [k[1] for k in list_tuple_new_captions]

/home/fidelrio/.pyenv/versions/3.7.5/envs/systematicity/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/fidelrio/.pyenv/versions/3.7.5/envs/systematicity/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [12]:
import spacy 
nlp = spacy.load("en_core_web_sm")

def find_prompt(x):
    """
    """
    prompt_segmentation = []
    doc = nlp(x['caption'])
    for wd in x['word_detected']:
        list_candidates = [k.text for k in doc.noun_chunks if wd in k.text.lower()]
        if len(list_candidates) > 1:
            prompt_segmentation.append('')
        elif len(list_candidates) == 0:
            prompt_segmentation.append(wd)
        else:
            prompt_segmentation.append(list_candidates[0])
    return prompt_segmentation

In [13]:
dftskill['prompt_segmentation'] = dftskill.apply(find_prompt, axis=1)

/home/fidelrio/.pyenv/versions/3.7.5/envs/systematicity/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [14]:
# Only keep when the noun phrase has been detected
dftskill = dftskill[dftskill['prompt_segmentation'].map(lambda x: np.mean([len(k) > 0 for k in x]) == 1)]

In [15]:
dftskill

,image_ID,caption_ID,caption,split,word_detected,augmented_captions,prompt_segmentation
336,1018148011,336,Men are standing on and about a truck carrying...,val,[men],[Women are standing on and about a truck carry...,[Men]
338,1018148011,338,A group of men are loading cotton onto a truck,val,[men],[A group of women are loading cotton onto a tr...,[men]
496,1029450589,496,A woman in black pants napping on the couch.,val,[woman],[A man in black pants napping on the couch.],[A woman]
497,1029450589,497,A man sleeping in a green room on a couch.,val,[man],[A woman sleeping in a green room on a couch.],[A man]
498,1029450589,498,The young woman is sleeping in her room.,val,[woman],[The young man is sleeping in his room.],[The young woman]
...,...,...,...,...,...,...,...
154859,981727078,154859,Two men using a pay phone.,val,[men],[Two women using a pay phone.],[Two men]
154995,991459823,154995,"A woman in a printed red dress, and brown sand...",val,[woman],"[A man in a printed red dress, and brown sanda...",[A woman]
154996,991459823,154996,"A man is exiting a port-a-potty, relieved he c...",val,[man],"[A woman is exiting a port-a-potty, relieved h...",[A man]
154997,991459823,154997,Two women wearing red and a man coming out of ...,val,"[man, women]",[Two women wearing red and a woman coming out ...,"[a man, Two women]"


In [16]:
dftskill['sample'] = False
k_to_sample = 300
selected_indices = np.random.choice(len(dftskill), k_to_sample, replace=False)
dftskill['sample'].iloc[selected_indices] = True

/home/fidelrio/.pyenv/versions/3.7.5/envs/systematicity/lib/python3.7/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [17]:
dftskill.to_csv('Image_quality_%s_%s.tsv'%(split, skill), sep='\t', index=False)
# files.download('Caption_training_%s.tsv'%skill)

In [18]:
assert False, "You shall not pass"

AssertionError: You shall not pass

In [ ]:
dftest = df[df.split == 'test']

for sk in ['gender', 'color', 'emotion', 'counting']:
  capts, _ = checker.find_captions_skill(list_captions=dftest.caption, skill=sk)
  dftestskill = dftest[dftest.caption.isin(capts)]
  dftestskill.to_csv('Caption_testing_%s.tsv'%sk, sep='\t', index=False)
  files.download('Caption_testing_%s.tsv'%sk)